In [1]:
"https://api.scryfall.com/cards/search?order=set&unique=art&q=set%3A'eld'+lang%3A'en'"

"https://api.scryfall.com/cards/search?order=set&unique=art&q=set%3A'eld'+lang%3A'en'"

In [2]:
# Change dir to get SQL connector
import os
os.chdir("..")
from src.sqldb import SQLDatabase

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
import seaborn as sns

# Updating Tables

In [4]:
# Constants
URL = "https://www.mtgtop8.com/format?f=ST"

In [5]:
from src import models as m
def update(URL):
    updater = m.Updater()
    updater.update_events(URL)
    updater.update_decks_and_players()
    updater.update_deck_lists()

In [98]:
update(URL)

Updating next page...
updating https://www.mtgtop8.com/event?e=29160&f=ST
updating https://www.mtgtop8.com/event?e=29161&f=ST


updater = m.Updater()
updater.update_cards()

# Cleaning

Loading data objects...

In [99]:
with SQLDatabase() as sql_db:
    event_table = sql_db.get_dataframe_from("event")
    deck_table = sql_db.get_dataframe_from("deck")
    deck_list_table = sql_db.get_dataframe_from("deckList")
    card_table = sql_db.get_dataframe_from("card")
    pilot_table = sql_db.get_dataframe_from("pilot")

# Clean Dates
event_table["date"] = pd.to_datetime(event_table["date"], dayfirst=True)

# Renaming ID Columns
event_table.rename(columns={"id": "eventId"}, inplace=True)
deck_table.rename(columns={"id": "deckId"}, inplace=True)
pilot_table.rename(columns={"id": "pilotId"}, inplace=True)

# Adding easier ID column to card_table
card_table["setNumber"] = card_table["setNumber"].str.zfill(3)
card_table["cardId"] = card_table["setNumber"] + card_table["setName"]

deck_table.loc[deck_table.deckId==601, "name"] = "GW Adventures"
deck_table.loc[deck_table.deckId==930, "name"] = "Izzet Mill"
deck_table.loc[deck_table.deckId==931, "name"] = "Sultai Ramp"
deck_table.loc[deck_table.deckId==932, "name"] = "Orzhov Aristocrats"
deck_table.loc[deck_table.deckId==934, "name"] = "Temur Control"

Adding archetypes to Deck Table.

These are important to see how play styles are distributed across standard.

Most general archetypes should be on top, so that the more specific archetypes get assigned if applicable.

In [100]:
# Adding archetypes to deck_table
archetypes = {
    "jund": ["jund"],
    "golgari": ["golgari"],
    "aggro": ["aggro", "red deck wins", "weenie", "ww", "mono green", "rdw", "devotion to black", "agro"],
    "control": ["control", "doom", "4cc", "yorion"],
    "midrange": ["midrange", "mid range"],
    "gyruda": ["gyruda"],
    "winota": ["winota"],
    "mill": ["mill"],
    "ramp": ["ramp", "omnath", "ultimatum", "forsaken monument"],
    "rogue": ["rogue"],
    "flash": ["flash"],
    "adventure": ["adventure"],
    "food": ["food"],
    "reclamation": ["reclamation"],
    "sacrifce": ["sacrifice", "aristocrats"],
    "cycling": ["cycling"],
    "kicker": ["kicker", "kick"],
    "mutate": ["mutate"],
    "stompy": ["stompy"],
    "dragons": ["dragon"]
}

for archetype in archetypes:
    for placeholder in archetypes[archetype]:
        deck_table.loc[
            (deck_table.name.str.lower().str.contains(placeholder)),
            "archetype"
        ] = archetype

deck_table.loc[deck_table.name == "5c Sanctum", "archetype"] = "control"
deck_table.loc[deck_table.deckId == 2452, "archetype"] = "aggro"
deck_table.loc[deck_table.deckId == 2757, "archetype"] = "control"
deck_table.loc[deck_table.deckId == 2975, "archetype"] = "aggro"
deck_table.loc[deck_table.deckId == 3020, "archetype"] = "aggro"
deck_table.loc[deck_table.deckId == 3024, "archetype"] = "aggro"

Adding color categories to Deck Table.

Color category is a big factor when looking at how the makeup of standard has evolved.

In [101]:
# Adding easy color groups to deck_table
categories = {
    "mono white": ["mono white", "mono-white", "white weenie", "weenie white", "ww"],
    "mono blue": ["mono blue", "mono-blue"],
    "mono black": ["mono black", "mono-black", "devotion to black"],
    "mono red": ["mono red", "mono-red", "red deck wins", "rdw"],
    "mono green": ["mono green", "mono-green", "monogreen"],
    "azorius": ["azorius", "uw", "wu"],
    "orzhov": ["orzhov", "wb", "bw"],
    "boros": ["boros", "wr", "rw", "winota"],
    "selesnya": ["selesnya", "wg", "gw"],
    "dimir": ["dimir", "ub"],
    "izzet": ["izzet", "ur"],
    "simic": ["simic", " ug "],
    "rakdos": ["rakdos", "br", "rb"],
    "golgari": ["golgari", "bg", "gb"],
    "gruul": ["gruul", "rg"],
    "bant": ["bant", "gwu", "guw", "ugw", "uwg", "wgu", "wug"],
    "esper": ["esper", "wub"],
    "grixis": ["grixis"],
    "jund": ["jund"],
    "naya": ["naya"],
    "jeskai": ["jeskai"],
    "sultai": ["sultai"],
    "mardu": ["mardu", "kroxa doom"],
    "temur": ["temur"],
    "abzan": ["abzan"],
    "4-color": ["omnath", "4c"],
    "colorless": ["forsaken monument"],
    "5-color": ["5c"]
}

for category in categories:
    for placeholder in categories[category]:
        deck_table.loc[
            (deck_table.name.str.lower().str.contains(placeholder)),
            "category"
        ] = category

deck_table.loc[
    (deck_table.name.str.lower() == "gyruda"),
    "category"
] = "4-color"

deck_table.loc[(deck_table.deckId == 360), "category"] = "gruul"

Some codes are not in Scryfall's api, and some (like the core sets) are not picked up in the source code, which is itself a bug. These codes are manually updated to equivalent cards with recognizable card codes.

In [102]:
# Replace all the abu codes with updated codes for basic lands 
# and Disenchant & Unsummon & Righteousness
abu_codes = {
    "092abu": "266eld",
    "170abu": "262eld",
    "130abu": "254eld",
    "196abu": "250eld",
    "249abu": "258eld",
    "065abu": "010znr",
    "271abu": "078m20",
    "215abu": "027eld"
}

# Adding randomly misformed codes.
other_codes = {
    "171cmd": "204m21", # Scavenging Ooze
    "356eld": "101eld", # Rankle, Master of Pranks
    "367eld": "147eld", # Torbran, Thane of Red Fell
    "335eld": "008eld", # Charming Prince
    "346eld": "054eld", # Midnight Clock
    "001unk": "", # Unknown card
    "003ivg": "174grn", # Goblin Electromancer
    "00410m": "004m10", # Baneslayer Angel
    "00514c": "013m21", # Containment Priest
    "021sha": "021shm", # Runed Halo
    "036str": "126eld", # Fling
    "0496th": "032m20", # Pacifism
    "05110m": "051m10", # Essence Scatter
    "05412m": "054m12", # Frost Breath
    "054m12 ": "054m12",
    "079sta": "103m21", # Grim Tutor
    "083urs": "096m21", # Duress
    "10113m": "101m13", # Murder
    "107dar": "241m21", # Tormod's Crypt
    "1136th": "159m21", # Shock
    "12410m": "124m10", # Act of Treason
    "1276th": "171m21", # Volcanic Geyser
    "144gui": "248rna", # Godless Shrine
    "162gui": "257grn", # Steam Vents
    "163gui": "259rna", # Stomping Ground
    "23215m": "245m15", # Radiant Fountain
    "237urs": "063m21", # Rewind
    "309thb": "076thb", # Thryx, the Sudden Storm
    "328iko": "162iko", # Kogla, the Titan Ape
    "334eld": "001eld", # Acclaimed Contender
    "364znr": "215znr",  # Turntimber Symbiosis // Turntimber, Serpentine...
    "117urs": "021m21",
    "189eld": "187eld",
    "012urs": "042khm"
}

card_table["cardId"].replace(to_replace=abu_codes, inplace=True)
deck_list_table["cardId"].replace(to_replace=abu_codes, inplace=True)

deck_list_table["cardId"].replace(to_replace=other_codes, inplace=True)



New decks need to be checked for misformed codes & missing archetypes/categories

In [103]:
deck_table.loc[deck_table.archetype.isna()]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category


In [104]:
deck_table.loc[deck_table.category.isna()]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category


In [105]:
unmatched = set(deck_list_table.loc[~(deck_list_table.cardId.isin(card_table.cardId))]["cardId"])

def find_urls_with_unmatched(unmatched):
    unmatched_deck_ids = list(deck_list_table.loc[deck_list_table.cardId == unmatched]["deckId"])
    return set(deck_table.loc[deck_table.deckId.isin(unmatched_deck_ids)]["deckUrl"])

def print_page_source(unmatched):
    unmatched_url = find_urls_with_unmatched(unmatched).pop()
    print(unmatched_url)
    html = requests.get("https://www.mtgtop8.com/event" + unmatched_url)
    return BeautifulSoup(html.text, features="lxml")

In [106]:
unmatched

{''}

In [107]:
ZENDIKAR_RELEASE = "2020-09-25"
KALDHEIM_RELEASE = "2021-02-05"

event_table.loc[event_table["date"] < ZENDIKAR_RELEASE, "latest_set"] = "Ikoria"
event_table.loc[(event_table["date"] >= ZENDIKAR_RELEASE) & (event_table["date"] < KALDHEIM_RELEASE), "latest_set"] = "Zendikar Rising"
event_table.loc[event_table["date"] >= KALDHEIM_RELEASE, "latest_set"] = "Kaldheim"

In [108]:
deck_table.loc[(deck_table["rank"] == "11 pts") & (deck_table["eventId"] == 22), "rank"] = 1
deck_table.loc[(deck_table["rank"] == "9 pts") & (deck_table["eventId"] == 22), "rank"] = 2
deck_table.loc[(deck_table["rank"] == "8 pts") & (deck_table["eventId"] == 22), "rank"] = 3
deck_table.loc[(deck_table["rank"] == "7 pts") & (deck_table["eventId"] == 22), "rank"] = 4
deck_table.loc[(deck_table["rank"] == "6 pts") & (deck_table["eventId"] == 22), "rank"] = 5
deck_table.loc[(deck_table["rank"] == "5 pts") & (deck_table["eventId"] == 22), "rank"] = 6
deck_table.loc[(deck_table["rank"] == "4 pts") & (deck_table["eventId"] == 22), "rank"] = 7

deck_table.loc[(deck_table["rank"] == "9 pts") & (deck_table["eventId"] == 30), "rank"] = 1
deck_table.loc[(deck_table["rank"] == "8 pts") & (deck_table["eventId"] == 30), "rank"] = 2
deck_table.loc[(deck_table["rank"] == "7 pts") & (deck_table["eventId"] == 30), "rank"] = 3
deck_table.loc[(deck_table["rank"] == "6 pts") & (deck_table["eventId"] == 30), "rank"] = 4
deck_table.loc[(deck_table["rank"] == "5 pts") & (deck_table["eventId"] == 30), "rank"] = 5

deck_table.loc[(deck_table["rank"] == "21 pts") & (deck_table["eventId"] == 300), "rank"] = 1
deck_table.loc[(deck_table["rank"] == "1 pts") & (deck_table["eventId"] == 300), "rank"] = 2

deck_table.loc[(deck_table["rank"] == "24 pts") & (deck_table["eventId"] == 408), "rank"] = 1
deck_table.loc[(deck_table["rank"] == "21 pts") & (deck_table["eventId"] == 408), "rank"] = 2
deck_table.loc[(deck_table["rank"] == "18 pts") & (deck_table["eventId"] == 408), "rank"] = 3

deck_table.loc[(deck_table["rank"] == "5-8"), "rank"] = 5

deck_table.loc[(deck_table["rank"] == "3-4"), "rank"] = 3

In [109]:
events_and_decks = pd.merge(
    event_table, 
    deck_table, 
    on="eventId",
    suffixes=["_event", "_deck"]
).drop_duplicates()

events_decks_pilots = pd.merge(
    events_and_decks,
    pilot_table,
    on="pilotId",
    suffixes=[None, "_pilot"]
).drop_duplicates()

full_table_no_cards = pd.merge(
    events_decks_pilots,
    deck_list_table,
    on="deckId",
    suffixes=[None, "_decklist"]
).drop_duplicates()

full_table1 = pd.merge(
    full_table_no_cards,
    card_table,
    on="cardId",
    suffixes=[None, "_cards"]
)[[
    "eventId",
    "name_event",
    "date",
    "deckId",
    "pilotId",
    "name_deck",
    "firstName",
    "lastName",
    "cardId",
    "name",
    "count",
    "color",
    "slot",
    "archetype",
    "category",
    "latest_set"
]].drop_duplicates().sort_values(by=["eventId", "deckId", "slot"]).reset_index(drop=True)

In [110]:
def update_set_codes(old, new, name=None, color=None):
    if name:
        full_table1.loc[full_table1["cardId"] == old, "name"] = name
    if color:
        full_table1.loc[full_table1["cardId"] == old, "color"] = color
    full_table1.loc[full_table1["cardId"] == old, "cardId"] = new
    deck_list_table.loc[deck_list_table["cardId"] == old, "cardId"] = new


update_set_codes("228ths", "256m21")
update_set_codes("043mor", "071znr")
update_set_codes("001frf", "001m21")
update_set_codes("119akh", "107iko")
update_set_codes("228roe", "247iko")
update_set_codes("201isd", "199m21")
update_set_codes("245mrd", "239m21")
update_set_codes("096isd", "083iko")
update_set_codes("005jou", "004thb")
update_set_codes("148rix", "209m21")
update_set_codes("225ths", "245thb")
update_set_codes("051m10", "049iko")
update_set_codes("246ktk", "257iko")
update_set_codes("165jou", "253m21")
update_set_codes("160bng", "246thb", "Temple of Enlightenment", "")
update_set_codes("161bng", "247thb", "Temple of Malice", "")
update_set_codes("088ths", "099thb", "Gray Merchant of Asphodel", "B")
update_set_codes("162bng", "248thb", "Temple of Plenty", "")
update_set_codes("175war", "197thb")
update_set_codes("169inv", "059m21", "Opt", "U")
update_set_codes("232ktk", "246iko")
update_set_codes("235ktk", "249iko")
update_set_codes("065mh1", "061m21")
update_set_codes("048zen", "062znr")
update_set_codes("227ths", "255m21")
update_set_codes("168zen", "193znr")
update_set_codes("065som", "102m21")
update_set_codes("168m11", "177m21")
update_set_codes("109aer", "188m21")
update_set_codes("115ori", "103eld")
update_set_codes("004m10", "006m21")
update_set_codes("226ths", "254m21")
update_set_codes("248xln", "233eld")
update_set_codes("164jou", "252m21")
update_set_codes("143isd", "145m21")
update_set_codes("043hou", "064m21")
update_set_codes("224ths", "244thb")
update_set_codes("088hou", "140m21")
update_set_codes("018som", "034thb")
update_set_codes("145m20", "141thb")
update_set_codes("190m11", "169iko")
update_set_codes("229ktk", "243iko")
update_set_codes("240ktk", "252iko")
update_set_codes("176war", "207m21")
update_set_codes("110xln", "107m21")
update_set_codes("046mbs", "114m21")
update_set_codes("201chk", "173m21")
update_set_codes("236ktk", "255iko", "Swiftwater Cliffs", "")
update_set_codes("237ktk", "256iko", "Thornwood Falls", "")
update_set_codes("255xln", "242thb", "Field of Ruin", "")
update_set_codes("232m15", "248m21", "Radiant Fountain", "")
update_set_codes("229ths", "249thb")
update_set_codes("190som", "234m21")
update_set_codes("010war", "009iko")
update_set_codes("079emn", "082m21")
update_set_codes("010ktk", "019m21")
update_set_codes("040ori", "043m21")
update_set_codes("223soi", "199znr")
update_set_codes("082mbs", "163iko")
update_set_codes("247ktk", "258iko")
update_set_codes("147roe", "147m21")
update_set_codes("078aer", "141m21")
update_set_codes("032m20", "025iko")
update_set_codes("016rav", "017m21")
update_set_codes("231ktk", "244iko")
update_set_codes("035m19", "031m21")
update_set_codes("145war", "161m21")
update_set_codes("041rav", "045iko")
update_set_codes("133ths", "149thb")
update_set_codes("168akh", "157iko")
update_set_codes("054m12", "051m21")
update_set_codes("245m19", "238m21")
update_set_codes("242ktk", "254iko")
update_set_codes("279leg", "121thb", "Underworld Dreams", "B")
update_set_codes("011dom", "016m21", "Dub", "W")
update_set_codes("310ice", "382znr", "Swamp", "")
update_set_codes("189dom", "142iko", "Adventurous Impulse", "G")
update_set_codes("076dom", "082iko", "Dark Bargain", "B")
update_set_codes("101m13", "109m20")
update_set_codes("172dis", "246rna")
update_set_codes("051tsp", "046m21")
update_set_codes("149dka", "227m20")
update_set_codes("021shm", "032m21")
update_set_codes("034isd", "042znr", "Smite the Monstrous","W")
update_set_codes("108kld", "110iko", "Cathartic Reunion", "R")
update_set_codes("090dom", "094m21", "Deathbloom Thallid", "B")
update_set_codes("037ktk", "054khm")
update_set_codes("307ice", "278khm", "Snow-Covered Island")
update_set_codes("308ice", "283khm", "Snow-Covered Mountain")
update_set_codes("306ice", "284khm", "Snow-Covered Forest")
update_set_codes("309ice", "277khm", "Snow-Covered Plains")
update_set_codes("075war", "085m21")
update_set_codes("100kld", "128znr")
update_set_codes("007ktk", "015m21")

In [112]:
full_table1.to_csv("flat_files/full_table.csv", index=False)
event_table.to_csv("flat_files/event_table.csv", index=False)
deck_table.to_csv("flat_files/deck_table.csv", index=False)
deck_list_table.to_csv("flat_files/deck_list_table.csv", index=False)
card_table.to_csv("flat_files/card_table.csv", index=False)
pilot_table.to_csv("flat_files/pilot_table.csv", index=False)

In [111]:
standard_sets = "eld|thb|znr|iko|khm|m21"
full_table1[
    (~full_table1["cardId"].str.contains(standard_sets, regex=True))
    & (full_table1["date"] >= "2020-09-29")
][["cardId", "name"]].drop_duplicates()

,cardId,name
9598,077grn,Midnight Reaper
14906,109m20,Murder


In [91]:
deck_table[
    deck_table["deckId"].isin(
        full_table1[
            full_table1["cardId"] == "309ice"
        ]
        ["deckId"]
    )
]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category


In [94]:
card_table[card_table["name"] == "Defiant Strike"]

,setNumber,setName,name,cmc,color,mana_cost,standardLegality,oracle_text,cardId
1153,015,m21,Defiant Strike,1,W,{W},legal,Target creature gets +1/+0 until end of turn.\...,015m21
1935,009,war,Defiant Strike,1,W,{W},legal,Target creature gets +1/+0 until end of turn.\...,009war
8675,007,ktk,Defiant Strike,1,W,{W},legal,Target creature gets +1/+0 until end of turn.\...,007ktk
